In [1]:
import utils

In [2]:
labels="'earthquake', 'quake', 'temblor', 'seism', 'tremor'"
time=True
participants=False
location=True
strictLocations=True

In [3]:
query=utils.get_me_a_query(labels, time, location, participants, strictLocations)
print(query)


	PREFIX sem: <http://semanticweb.cs.vu.nl/2009/11/sem/>
	PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
	PREFIX dbpedia: <http://dbpedia.org/resource/>
	PREFIX owltime: <http://www.w3.org/TR/owl-time#>
	PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
	PREFIX dbo: <http://dbpedia.org/ontology/>
	SELECT ?event ?location ?time ?participant WHERE {
    
		?event a sem:Event .
		{ ?event rdfs:label ?label .
		FILTER (?label IN ('earthquake', 'quake', 'temblor', 'seism', 'tremor')) } 
		UNION
		{ ?event sem:hasActor ?nonentity .
		?nonentity skos:relatedMatch dbpedia:Earthquake } . 
		?event sem:hasTime ?tmx . ?tmx owltime:inDateTime ?time .  
		?event sem:hasPlace ?location .  
		?location a ?x . ?x rdfs:subClassOf* dbo:Place .  
    
	}
    


In [4]:
res=utils.get_sparql_results(query)
len(res)

234

In [5]:
print(res)

[{'time': {'value': 'http://www.newsreader-project.eu/time/20150923', 'type': 'uri'}, 'location': {'value': 'http://dbpedia.org/resource/Northamptonshire', 'type': 'uri'}, 'event': {'value': 'http://www.newsreader-project.eu/data/wikinews/sigmed/data/39b7172f-9b21-403c-ba51-548c1dfa0a65.in.naf#ev11', 'type': 'uri'}}, {'time': {'value': 'http://www.newsreader-project.eu/time/20150923', 'type': 'uri'}, 'location': {'value': 'http://dbpedia.org/resource/Northamptonshire', 'type': 'uri'}, 'event': {'value': 'http://www.newsreader-project.eu/data/wikinews/sigmed/data/46af2648-e0af-4fb5-abac-590c634b8a8f.in.naf#ev11', 'type': 'uri'}}, {'time': {'value': 'http://www.newsreader-project.eu/time/20150923', 'type': 'uri'}, 'location': {'value': 'http://dbpedia.org/resource/Northamptonshire', 'type': 'uri'}, 'event': {'value': 'http://www.newsreader-project.eu/data/wikinews/sigmed/data/69759970-5851-451c-b1b9-09d999590d59.in.naf#ev9', 'type': 'uri'}}, {'time': {'value': 'http://www.newsreader-proj